In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

In [169]:
from sklearn import model_selection
from sklearn import preprocessing

In [197]:
train_data=pd.read_csv("data/train.csv")
test_data=pd.read_csv("data/test.csv")

# Preprocessing

In [199]:
print train_data.shape
print test_data.shape

(891, 12)
(418, 11)


##### Filling the Null value for train and test data 

In [200]:
print "Train columns with null values:" 
print train_data.isnull().sum()

Train columns with null values:
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64


In [201]:
print "Test columns with null values:" 
print test_data.isnull().sum()

Test columns with null values:
PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


##### Variable Tramsformation 

In [203]:
all_data=[train_data,test_data]
for dataset in all_data:
    dataset["Age"].fillna(dataset['Age'].mean(), inplace = True)
    dataset["Fare"].fillna(dataset['Fare'].median(), inplace = True)
    dataset["Embarked"].fillna(dataset['Embarked'].mode(), inplace = True)

In [204]:
train_data.drop(['PassengerId','Cabin', 'Ticket'],axis=1,inplace=True)

In [205]:
train_data.describe(include="all")

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked
count,891.000000,891.000000,891,891,891.000000,891.000000,891.000000,891.000000,889
unique,NaN,NaN,891,2,NaN,NaN,NaN,NaN,3
top,NaN,NaN,"Graham, Mr. George Edward",male,NaN,NaN,NaN,NaN,S
freq,NaN,NaN,1,577,NaN,NaN,NaN,NaN,644
mean,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,32.204208,NaN
std,0.486592,0.836071,NaN,NaN,13.002015,1.102743,0.806057,49.693429,NaN
min,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,0.000000,NaN
25%,0.000000,2.000000,NaN,NaN,22.000000,0.000000,0.000000,7.910400,NaN
50%,0.000000,3.000000,NaN,NaN,29.699118,0.000000,0.000000,14.454200,NaN
75%,1.000000,3.000000,NaN,NaN,35.000000,1.000000,0.000000,31.000000,NaN


In [206]:
for dataset in all_data:
    dataset["familySize"]=dataset["SibSp"]+dataset["Parch"]+1
    dataset["isAlone"]=1
    dataset["isAlone"].loc[dataset['familySize'] > 1] = 0
    dataset["title"]=dataset.apply(lambda x: x["Name"].split(",")[1].split(".")[0].strip(" \n"),axis=1)
    dataset["FareBin"]=pd.qcut(dataset['Fare'], 4)
    dataset["AgeBin"]=pd.cut(dataset["Age"],5)

In [207]:
train_data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,familySize,isAlone,title,FareBin,AgeBin
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,2,0,Mr,"[0, 7.91]","(16.336, 32.252]"
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,2,0,Mrs,"(31, 512.329]","(32.252, 48.168]"
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,1,1,Miss,"(7.91, 14.454]","(16.336, 32.252]"
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,2,0,Mrs,"(31, 512.329]","(32.252, 48.168]"
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,1,1,Mr,"(7.91, 14.454]","(32.252, 48.168]"


In [208]:
train_data["title"].loc[(train_data["title"]!="Mr") &
                       (train_data["title"]!="Miss") &
                       (train_data["title"]!="Mrs") &
                       (train_data["title"]!="Master")]="Other"

In [209]:
train_data["title"].value_counts()

Mr        517
Miss      182
Mrs       125
Master     40
Other      27
Name: title, dtype: int64

In [224]:
#original variables x
x_vals=['Sex','Pclass', 'Embarked', 'title','SibSp', 'Parch', 'Age', 'Fare', 'familySize', 'isAlone']

#Target Y
target=["Survived"]

#Code categorical data
#Transform categorical data into numerical value
label=preprocessing.LabelEncoder()
for dataset in all_data:
    dataset["sexCode"]=label.fit_transform(dataset["Sex"])
    dataset["embarkedCode"]=label.fit_transform(dataset["Embarked"])
    dataset["titleCode"]=label.fit_transform(dataset["title"])
    dataset["ageCode"]=label.fit_transform(dataset["AgeBin"])
    dataset["fareCode"]=label.fit_transform(dataset["FareBin"])

raw_x_vals = ['sexCode','Pclass', 'embarkedCode', 'titleCode','SibSp', 'Parch', 'Age', 'Fare']    

bin_x_vals=['sexCode','Pclass', 'embarkedCode', 'titleCode', 'ageCode', 
            'fareCode', 'familySize']

dummy_data=pd.get_dummies(train_data[x_vals])
dummy_x_vals=dummy_data.columns.tolist()


In [225]:
print raw_x_vals
print bin_x_vals
print dummy_x_vals

['sexCode', 'Pclass', 'embarkedCode', 'titleCode', 'SibSp', 'Parch', 'Age', 'Fare']
['sexCode', 'Pclass', 'embarkedCode', 'titleCode', 'ageCode', 'fareCode', 'familySize']
['Pclass', 'SibSp', 'Parch', 'Age', 'Fare', 'familySize', 'isAlone', 'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S', 'title_Master', 'title_Miss', 'title_Mr', 'title_Mrs', 'title_Other']


In [226]:
dummy_data.head()

,Pclass,SibSp,Parch,Age,Fare,familySize,isAlone,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,title_Master,title_Miss,title_Mr,title_Mrs,title_Other
0,3,1,0,22.0,7.2500,2,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
1,1,1,0,38.0,71.2833,2,0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,3,0,0,26.0,7.9250,1,1,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,1,1,0,35.0,53.1000,2,0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,3,0,0,35.0,8.0500,1,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [219]:
train_data.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,familySize,isAlone,title,FareBin,AgeBin,sexCode,embarkedCode,titleCode,ageCode,fareCode
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,2,0,Mr,"[0, 7.91]","(16.336, 32.252]",1,3,2,1,3
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,2,0,Mrs,"(31, 512.329]","(32.252, 48.168]",0,1,3,2,1
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,1,1,Miss,"(7.91, 14.454]","(16.336, 32.252]",0,3,1,1,2
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,2,0,Mrs,"(31, 512.329]","(32.252, 48.168]",0,3,3,2,1
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,1,1,Mr,"(7.91, 14.454]","(32.252, 48.168]",1,3,2,2,2


##### Spliting data  

In [227]:
train_raw_x,test_raw_x,train_raw_y,test_raw_y=model_selection.train_test_split(train_data[raw_x_vals],
                                                                               train_data[target],random_state=0)

train_bin_x,test_bin_x,train_bin_y,test_bin_y=model_selection.train_test_split(train_data[bin_x_vals],
                                                                               train_data[target],random_state=0)
train_dum_x,test_dum_x,train_dum_y,test_dum_y=model_selection.train_test_split(dummy_data[dummy_x_vals],
                                                                               train_data[target],random_state=0)

In [229]:
print "raw train data shape : {}".format(train_data.shape)
print "train_raw shape : {}".format(train_raw_x.shape)
print "test_raw shape : {}".format(test_raw_x.shape)

raw train data shape : (891, 19)
train_raw shape : (668, 8)
test_raw shape : (223, 8)


In [151]:
train_data['title'][0]

'Mr'

In [6]:
train_data["Survived"].value_counts()

0    549
1    342
Name: Survived, dtype: int64

In [15]:
train_data.groupby(["Survived","Pclass"])["PassengerId"].count()


Survived  Pclass
0         1          80
          2          97
          3         372
1         1         136
          2          87
          3         119
Name: PassengerId, dtype: int64

In [7]:
train_data["Pclass"].value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [8]:
train_data["Sex"].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [9]:
train_data["SibSp"].value_counts()

0    608
1    209
2     28
4     18
3     16
8      7
5      5
Name: SibSp, dtype: int64

In [10]:
train_data["Parch"].value_counts()

0    678
1    118
2     80
5      5
3      5
4      4
6      1
Name: Parch, dtype: int64

In [11]:
train_data["Embarked"].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [24]:
train_data["Cabin"].value_counts()

C23 C25 C27        4
G6                 4
B96 B98            4
D                  3
C22 C26            3
E101               3
F2                 3
F33                3
B57 B59 B63 B66    2
C68                2
B58 B60            2
E121               2
D20                2
E8                 2
E44                2
B77                2
C65                2
D26                2
E24                2
E25                2
B20                2
C93                2
D33                2
E67                2
D35                2
D36                2
C52                2
F4                 2
C125               2
C124               2
                  ..
F G63              1
A6                 1
D45                1
D6                 1
D56                1
C101               1
C54                1
D28                1
D37                1
B102               1
D30                1
E17                1
E58                1
F E69              1
D10 D12            1
E50                1
A14          